In [ ]:
from datasets import concatenate_datasets


In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import librosa

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "/home/khj6051/whisper/whisper-finetune/checkpoint-43000"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained("openai/whisper-large-v3-turbo")

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

In [ ]:
from datasets import load_dataset

ds = load_dataset("kresnik/zeroth_korean")["train"]
splits = ds.train_test_split(test_size=0.1, seed=42)  # seed 고정하면 재현성 O

train_ds = splits["train"]
val_ds   = splits["test"]

In [ ]:
import torchaudio

for data in val_ds:
    array = torch.tensor(data['audio']['array']).to(device)
    sr = data['audio']['sampling_rate']
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000).to(device)
        array = resampler(array)
    result = pipe(array)
    print(result)
    print(data['text'])
    break

In [ ]:
from datasets import load_dataset
path = "Emilia-YODAS/KO/*.tar" # Same for Emilia-YODAS; just replace "Emilia/" with "Emilia-YODAS/"
dataset = load_dataset("amphion/Emilia-Dataset", data_files={"ko": path}, split="ko", streaming=False)
splits = dataset.train_test_split(test_size=0.1, seed=42)  # seed 고정하면 재현성 O

train_ds = splits["train"]
val_ds   = splits["test"].select(range(3000))

print(len(train_ds))

In [ ]:
import torch
device='cuda'
import torchaudio

for idx, data in enumerate(val_ds):
    array = torch.tensor(data['mp3']['array']).to(device)
    print(array.shape)
    if idx>10:
        break

In [ ]:
val_ds[0]

In [ ]:
import numpy as np
from datasets import load_dataset, Audio
from utils import clean_transcript

ds = load_dataset("idiotDeveloper/koreanTelephone")
ds = ds.cast_column("audio", Audio(decode=True, sampling_rate=16000))

print(len(ds['test']))
arr = np.array(ds['test']["transcripts"])
mask = np.array([not ("((" in t or "))" in t) for t in arr])
ds['test'] = ds['test'].select(np.where(mask)[0])
print(len(ds['test']))

ds['test'] = ds["test"].map(clean_transcript)

In [ ]:
print(len(ds['test']['transcripts']))
print(len(set(ds['test']['transcripts'])))

In [ ]:
import pandas as pd

df = ds["test"].to_pandas()
df = df.drop_duplicates(subset=["transcripts"])  # transcripts 기준 중복 제거

# 다시 Hugging Face Dataset으로 변환
from datasets import Dataset
ds["test"] = Dataset.from_pandas(df, preserve_index=False)
print(len(ds['test']['transcripts']))

In [1]:
# ------------------------------------
# 2) 텍스트 전처리(간단 버전: 한국어 위주)
# ------------------------------------
def normalize_text(txt: str) -> str:
    """
    - 소문자화, 양끝 공백 제거
    - 불필요한 중복 공백 제거
    - (옵션) 기호 단순화 등
    NOTE: 프로젝트 특성에 맞춰 적절히 확장하세요.
    """
    if txt is None:
        return ""
    t = txt.strip()
    # Whisper는 케이스/구두점 학습도 가능하므로, 너무 aggressive 하게 제거하지 않습니다.
    t = " ".join(t.split())
    return t

'Whisper는 케이스/구두점 학습도 가능하므로, 너무 aggressive 하게 제거하지 않습니다.'